<a href="https://colab.research.google.com/github/suraj93/Intro-to-Quant-Modelling-CFA-2025/blob/main/load_yahoo_finance_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls "/content/gdrive/My Drive"

'1022 Prestige White Meadows - floor plan 1.PNG'
'1022 Prestige White Meadows - floor plan 2.PNG'
'1 - vallabha.mp3'
'2022 Honeymoon Trip Bali Singapore.gsheet'
'2024 trips and budget.gsheet'
'2025 Laptop backup'
'2025 trips.gsheet'
'2 BHK 2174.xlsx'
'2 BHK 2174.xlsx.gsheet'
'2 - saraseeruha.mp3'
'3 - natajanapalini.mp3'
'4 - parvathi kumara.mp3'
'5 - nannu palimpa.mp3'
'6 - bhavayami.mp3'
'7 - thillana darbari kannada.mp3'
'8 - shanthi nilava vendum.mp3'
'9 - concluding remarks.mp3'
'Abirami Andhadhi'
'ACKO case study draft v2.docx'
'Adaptive Control'
'Advanced Topics in DSP.spd'
'Aero project ID 2011 rough.gdoc'
'All the clothes we own are not enough.gdoc'
'Andaman pics dump'
'Aru gs'
'Asset declaration-Format.xlsx'
'Avani Aavattam'
'baaro krishnaiyya Sanju video.mp4'
'Bay Area places.gdoc'
'best and worst circumstances - madras event 2_0.avi'
'Blaming the world will not help you - chennai event 1_0.avi'
'Blaming the world will not help you - madras event 2_0.avi'
'BMW EMI math.gshee

In [ ]:
import os
folder_path = '/content/gdrive/My Drive/Finance AI/2025 CFA Quant workshop/'
files_in_folder = os.listdir(folder_path)
print(files_in_folder)

['__pycache__', 'data', 'yfinance_writer.py', 'load_yahoo_finance_data.ipynb']


In [ ]:
%%writefile '/content/gdrive/My Drive/Finance AI/2025 CFA Quant workshop/yfinance_writer.py'

"""
India Market Data Downloader
=============================
Downloads historical price data for Indian ETFs and indices from Yahoo Finance
Formats data for use in the Portfolio Analysis Workshop

Author: Workshop Data Team
Date: 2024
"""

import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import numpy as np
import os

# ============================================================================
# CONFIGURATION
# ============================================================================

# Yahoo Finance tickers for Indian market
TICKERS = {
    # ETFs (NSE-listed)
    'NIFTYBEES': 'NIFTYBEES.NS',      # Nifty 50 ETF
    'JUNIORBEES': 'JUNIORBEES.NS',    # Nifty Next 50 (alternative to small cap)
    'BANKBEES': 'BANKBEES.NS',        # Bank Nifty ETF
    'GOLDBEES': 'GOLDBEES.NS',        # Gold ETF
    'LIQUIDBEES': 'LIQUIDBEES.NS',    # Liquid Fund ETF

    # Indices (for factor analysis)
    'NIFTY50': '^NSEI',               # Nifty 50 Index
    'NIFTY_BANK': '^NSEBANK',         # Bank Nifty Index
    'NIFTY_SMALLCAP': '^CNXSC',       # Small Cap Index (alternative ticker)
    'NIFTY_MIDCAP': '^NSEMDCP50',     # Mid Cap 50
    'NIFTY_VALUE': '^CNXVALUE',       # Value Index (may have limited data)
    'NIFTY_MOMENTUM': '^NIFMOM30',    # Momentum 30 (may have limited data)
}

# Alternative tickers if primary ones fail
ALTERNATIVE_TICKERS = {
    'NIFTYSMALL': 'NIFTYSMALL.NS',    # Small Cap ETF (if available)
    'KOTAKGOLD': 'KOTAKGOLD.NS',      # Alternative Gold ETF
    'SETFGOLD': 'SETFGOLD.NS',        # Another Gold ETF option
}

# Bond proxy - we'll create synthetic data as Indian bond ETFs have limited history
BOND_CONFIG = {
    'base_rate': 0.07,        # 7% base return (India 10Y Gsec average)
    'volatility': 0.05,       # 5% annual volatility
    'ticker_name': 'GSEC10YR'
}


# Date range
START_DATE = '2010-01-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')

# Output configuration
OUTPUT_DIR = 'data'
OUTPUT_FILE = 'india_market_data.csv'
OUTPUT_FILE = 'india_portfolios.csv'
METADATA_FILE = 'data_metadata_2.txt'

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def download_ticker(ticker, ticker_name, start_date, end_date, max_retries=3):
    """
    Download data for a single ticker with retry logic

    Parameters:
    -----------
    ticker : str
        Yahoo Finance ticker symbol
    ticker_name : str
        Friendly name for the ticker
    start_date : str
        Start date in YYYY-MM-DD format
    end_date : str
        End date in YYYY-MM-DD format
    max_retries : int
        Number of retry attempts

    Returns:
    --------
    pd.Series : Adjusted close prices, or None if download fails
    """
    for attempt in range(max_retries):
        try:
            print(f"  Downloading {ticker_name} ({ticker})... ", end='')

            # Download data
            data = yf.download(ticker, start=start_date, end=end_date,
                             progress=False)

            if data.empty:
                print(f"❌ No data available")
                return None

            # Use Adjusted Close to account for dividends and splits
            if 'Adj Close' in data.columns:
                prices = data['Adj Close']
            elif 'Close' in data.columns:
                prices = data['Close']
            else:
                print(f"❌ No price column found")
                return None

            # Handle multi-level columns (sometimes yfinance returns these)
            if isinstance(prices, pd.DataFrame):
                prices = prices.iloc[:, 0]

            # Clean data
            prices = prices.dropna()

            if len(prices) < 100:  # Minimum data threshold
                print(f"❌ Insufficient data ({len(prices)} days)")
                return None

            print(f"✅ {len(prices)} days")
            return prices

        except Exception as e:
            print(f"⚠️  Attempt {attempt + 1} failed: {str(e)}")
            if attempt == max_retries - 1:
                print(f"❌ Failed after {max_retries} attempts")
                return None

    return None


def create_synthetic_bond_data(dates, config):
    """
    Create synthetic bond price data based on typical India Gsec behavior

    Parameters:
    -----------
    dates : pd.DatetimeIndex
        Dates for which to generate data
    config : dict
        Configuration with base_rate and volatility

    Returns:
    --------
    pd.Series : Synthetic bond prices
    """
    np.random.seed(42)  # For reproducibility

    n_days = len(dates)
    annual_return = config['base_rate']
    annual_vol = config['volatility']

    # Calculate daily parameters
    daily_return = annual_return / 252
    daily_vol = annual_vol / np.sqrt(252)

    # Generate returns with mean reversion (bonds are mean-reverting)
    returns = np.random.normal(daily_return, daily_vol, n_days)

    # Add mean reversion (bonds tend to revert to long-term yields)
    for i in range(1, len(returns)):
        returns[i] += -0.1 * returns[i-1]  # Mean reversion coefficient

    # Calculate prices
    initial_price = 100
    prices = initial_price * np.cumprod(1 + returns)

    return pd.Series(prices, index=dates, name=config['ticker_name'])


def create_synthetic_liquid_fund(dates, annual_return=0.055):
    """
    Create synthetic liquid fund data (near-constant growth at money market rates)

    Parameters:
    -----------
    dates : pd.DatetimeIndex
        Dates for which to generate data
    annual_return : float
        Annual return (default 5.5% typical for liquid funds)

    Returns:
    --------
    pd.Series : Synthetic liquid fund prices
    """
    daily_return = annual_return / 365  # Simple interest for liquid funds
    initial_price = 1000  # NAV-like starting point

    cumulative_days = np.arange(len(dates))
    prices = initial_price * (1 + daily_return) ** cumulative_days

    # Add tiny bit of noise (liquid funds have minimal volatility)
    np.random.seed(42)
    noise = np.random.normal(0, 0.0001, len(dates))
    prices = prices * (1 + noise)

    return pd.Series(prices, index=dates, name='LIQUIDBEES')


def align_data(data_dict, method='ffill'):
    """
    Align all data series to common date index

    Parameters:
    -----------
    data_dict : dict
        Dictionary of ticker_name: pd.Series
    method : str
        How to handle missing dates ('ffill', 'bfill', 'drop')

    Returns:
    --------
    pd.DataFrame : Aligned price data
    """
    # Create DataFrame from dict
    df = pd.DataFrame(data_dict)

    # Forward fill missing values (common for Indian market holidays)
    if method == 'ffill':
        df = df.ffill()
    elif method == 'bfill':
        df = df.bfill()

    # Drop any remaining NaN rows
    df = df.dropna()

    return df


def calculate_data_quality_metrics(df):
    """
    Calculate data quality metrics for reporting

    Returns:
    --------
    dict : Quality metrics
    """
    metrics = {}

    for col in df.columns:
        series = df[col]

        # Calculate returns for quality checks
        returns = series.pct_change().dropna()

        metrics[col] = {
            'start_date': series.index[0].strftime('%Y-%m-%d'),
            'end_date': series.index[-1].strftime('%Y-%m-%d'),
            'total_days': len(series),
            'trading_days': len(returns),
            'missing_pct': (series.isna().sum() / len(series)) * 100,
            'start_price': series.iloc[0],
            'end_price': series.iloc[-1],
            'total_return_pct': ((series.iloc[-1] / series.iloc[0]) - 1) * 100,
            'mean_daily_return_pct': returns.mean() * 100,
            'daily_volatility_pct': returns.std() * 100,
            'max_drawdown_pct': ((series / series.cummax()) - 1).min() * 100,
        }

    return metrics


# ============================================================================
# MAIN DOWNLOAD FUNCTION
# ============================================================================

def download_all_data(*args, **kwargs):
    """
    Main function to download and process all market data
    """

    if 'inputfolder' in kwargs:
        inputfolder = kwargs['inputfolder']
    else:
        inputfolder = OUTPUT_DIR

    print("=" * 80)
    print("INDIA MARKET DATA DOWNLOADER")
    print("=" * 80)
    print(f"\nDate Range: {START_DATE} to {END_DATE}")
    print(f"Output Directory: {inputfolder}/")
    print("\n" + "-" * 80)
    print("DOWNLOADING DATA...")
    print("-" * 80 + "\n")



    # Create output directory
    os.makedirs(inputfolder, exist_ok=True)

    # Dictionary to store all downloaded data
    all_data = {}
    download_log = []

    # Download primary tickers
    print("Primary Tickers:")
    for ticker_name, ticker_symbol in TICKERS.items():
        prices = download_ticker(ticker_symbol, ticker_name, START_DATE, END_DATE)

        if prices is not None:
            all_data[ticker_name] = prices
            download_log.append({
                'ticker': ticker_name,
                'symbol': ticker_symbol,
                'status': 'SUCCESS',
                'days': len(prices)
            })
        else:
            download_log.append({
                'ticker': ticker_name,
                'symbol': ticker_symbol,
                'status': 'FAILED',
                'days': 0
            })

    print("\n" + "-" * 80)
    print("CREATING SYNTHETIC DATA (for assets with limited Yahoo data)...")
    print("-" * 80 + "\n")

    # Create common date range from successfully downloaded data
    if all_data:
        # Get the most complete date range
        all_indices = [series.index for series in all_data.values()]
        common_dates = all_indices[0]
        for idx in all_indices[1:]:
            common_dates = common_dates.union(idx)
        common_dates = common_dates.sort_values()
    else:
        # Fallback: create date range manually
        common_dates = pd.date_range(start=START_DATE, end=END_DATE, freq='D')

    # Create synthetic bond data
    print(f"  Creating {BOND_CONFIG['ticker_name']} (India 10Y Gsec proxy)... ", end='')
    bond_data = create_synthetic_bond_data(common_dates, BOND_CONFIG)
    all_data[BOND_CONFIG['ticker_name']] = bond_data
    print(f"✅ {len(bond_data)} days")
    download_log.append({
        'ticker': BOND_CONFIG['ticker_name'],
        'symbol': 'SYNTHETIC',
        'status': 'CREATED',
        'days': len(bond_data)
    })

    # Check if we need synthetic liquid fund (if LIQUIDBEES download failed)
    if 'LIQUIDBEES' not in all_data or all_data['LIQUIDBEES'] is None:
        print(f"  Creating LIQUIDBEES (Liquid Fund synthetic)... ", end='')
        liquid_data = create_synthetic_liquid_fund(common_dates)
        all_data['LIQUIDBEES'] = liquid_data
        print(f"✅ {len(liquid_data)} days")
        download_log.append({
            'ticker': 'LIQUIDBEES',
            'symbol': 'SYNTHETIC',
            'status': 'CREATED',
            'days': len(liquid_data)
        })

    print("\n" + "-" * 80)
    print("ALIGNING AND CLEANING DATA...")
    print("-" * 80 + "\n")

    # Align all data to common dates
    aligned_data = align_data(all_data, method='ffill')
    print(f"  ✅ Data aligned: {len(aligned_data)} trading days")
    print(f"  ✅ Assets available: {len(aligned_data.columns)}")

    # Calculate quality metrics
    print("\n" + "-" * 80)
    print("DATA QUALITY METRICS")
    print("-" * 80 + "\n")

    quality_metrics = calculate_data_quality_metrics(aligned_data)

    # Save to CSV
    output_path = os.path.join(inputfolder, OUTPUT_FILE)
    aligned_data.to_csv(output_path)
    print(f"  ✅ Data saved to: {output_path}")

    # Create metadata file
    metadata_path = os.path.join(inputfolder, METADATA_FILE)
    with open(metadata_path, 'w') as f:
        f.write("=" * 80 + "\n")
        f.write("INDIA MARKET DATA - METADATA\n")
        f.write("=" * 80 + "\n\n")

        f.write(f"Download Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Date Range: {START_DATE} to {END_DATE}\n")
        f.write(f"Total Trading Days: {len(aligned_data)}\n")
        f.write(f"Number of Assets: {len(aligned_data.columns)}\n\n")

        f.write("-" * 80 + "\n")
        f.write("DOWNLOAD LOG\n")
        f.write("-" * 80 + "\n\n")

        for log in download_log:
            f.write(f"{log['ticker']:20s} ({log['symbol']:20s}): {log['status']:10s} - {log['days']} days\n")

        f.write("\n" + "-" * 80 + "\n")
        f.write("QUALITY METRICS\n")
        f.write("-" * 80 + "\n\n")

        for ticker, metrics in quality_metrics.items():
            f.write(f"\n{ticker}:\n")
            f.write(f"  Period: {metrics['start_date']} to {metrics['end_date']}\n")
            f.write(f"  Trading Days: {metrics['trading_days']}\n")
            f.write(f"  Total Return: {metrics['total_return_pct']:.2f}%\n")
            f.write(f"  Annualized Return: {(metrics['total_return_pct'] / (metrics['trading_days']/252)):.2f}%\n")
            f.write(f"  Daily Volatility: {metrics['daily_volatility_pct']:.4f}%\n")
            f.write(f"  Annual Volatility: {metrics['daily_volatility_pct'] * np.sqrt(252):.2f}%\n")
            f.write(f"  Max Drawdown: {metrics['max_drawdown_pct']:.2f}%\n")

    print(f"  ✅ Metadata saved to: {metadata_path}")

    # Print summary statistics
    print("\n" + "=" * 80)
    print("DOWNLOAD SUMMARY")
    print("=" * 80 + "\n")

    summary_df = pd.DataFrame(download_log)
    print(summary_df.to_string(index=False))

    print("\n" + "=" * 80)
    print("DATA PREVIEW")
    print("=" * 80 + "\n")

    print(aligned_data.head(10))
    print("\n...")
    print(aligned_data.tail(10))

    print("\n" + "=" * 80)
    print("✅ DOWNLOAD COMPLETE!")
    print("=" * 80)
    print(f"\nFiles created:")
    print(f"  1. {output_path}")
    print(f"  2. {metadata_path}")
    print(f"\nReady to use in workshop notebooks!")

    return aligned_data, quality_metrics


# ============================================================================
# USAGE EXAMPLES
# ============================================================================

def load_workshop_data(filepath='data/india_market_data.csv'):
    """
    Convenience function to load data in workshop notebooks

    Usage in notebook:
    ------------------
    from download_india_market_data import load_workshop_data
    prices = load_workshop_data()
    """
    df = pd.read_csv(filepath, index_col=0, parse_dates=True)
    return df


def get_returns(prices, period='daily'):
    """
    Calculate returns from price data

    Parameters:
    -----------
    prices : pd.DataFrame
        Price data
    period : str
        'daily', 'weekly', 'monthly', 'annual'
    """
    if period == 'daily':
        return prices.pct_change()
    elif period == 'weekly':
        return prices.resample('W').last().pct_change()
    elif period == 'monthly':
        return prices.resample('M').last().pct_change()
    elif period == 'annual':
        return prices.resample('A').last().pct_change()
    else:
        raise ValueError("Period must be 'daily', 'weekly', 'monthly', or 'annual'")


# ============================================================================
# RUN SCRIPT
# ============================================================================
'''
if __name__ == "__main__":
    # Download all data
    data, metrics = download_all_data()

    print("\n" + "=" * 80)
    print("NEXT STEPS")
    print("=" * 80)
    print("\nIn your Jupyter notebook, load data with:")
    print("\n  import pandas as pd")
    print("  prices = pd.read_csv('data/india_market_data.csv', index_col=0, parse_dates=True)")
    print("\n  # Or use the helper function:")
    print("  from download_india_market_data import load_workshop_data")
    print("  prices = load_workshop_data()")
    print("\n" + "=" * 80)
'''

Overwriting /content/gdrive/My Drive/Finance AI/2025 CFA Quant workshop/yfinance_writer.py


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import sys
sys.path.append('/content/gdrive/My Drive/Finance AI/2025 CFA Quant workshop/')

!ls "/content/gdrive/My Drive/Finance AI/2025 CFA Quant workshop"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
data  load_yahoo_finance_data.ipynb  __pycache__  yfinance_writer.py


In [ ]:
import yfinance_writer as yfw
import os
import importlib

directory = '/content/gdrive/My Drive/Finance AI/2025 CFA Quant workshop/data'
if not os.path.exists(directory):
    os.makedirs(directory)

# Reload the module to ensure the latest function definition is used
importlib.reload(yfw)

# yfw.download_all_data(inputfolder='/content/gdrive/My Drive/Finance AI/2025 CFA Quant workshop/data/')


<module 'yfinance_writer' from '/content/gdrive/My Drive/Finance AI/2025 CFA Quant workshop/yfinance_writer.py'>

In [ ]:
p1 = yfw.download_ticker("^CNXFMCG","NIFTY_FMCG" , yfw.START_DATE , yfw.END_DATE)
#yfw.download_ticker("NIFTYSMLCAP250.NS","NIFTY_SMALLCAP" , yfw.START_DATE , yfw.END_DATE)

p1.to_csv("a1.csv")


/content/gdrive/My Drive/Finance AI/2025 CFA Quant workshop/yfinance_writer.py:93: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date,


In [ ]:
import yfinance_writer as yfw
inputfolder = "/content/gdrive/My Drive/Finance AI/2025 CFA Quant workshop/data/"
aa = yfw.load_workshop_data(inputfolder+'india_market_data.csv')

In [ ]:
aa

,NIFTYBEES,JUNIORBEES,BANKBEES,GOLDBEES,LIQUIDBEES,NIFTY50,NIFTY_BANK,NIFTY_MIDCAP,GSEC10YR
Date,,,,,,,,,
2010-01-04,52.393002,105.849998,91.699997,16.549900,589.801880,5232.200195,9112.244141,2689.708740,100.184228
2010-01-05,52.896000,106.550003,92.000000,16.680000,589.801880,5277.899902,9192.043945,2744.155762,100.149971
2010-01-06,53.087002,108.150002,91.300003,16.600000,589.801880,5281.799805,9222.892578,2759.804932,100.385523
2010-01-07,52.806000,107.349998,91.000000,16.572001,589.807068,5263.100098,9192.843750,2741.105957,100.871356
2010-01-08,52.584999,105.550003,91.900002,16.490000,589.801880,5244.750000,9160.593750,2744.005615,100.776164
...,...,...,...,...,...,...,...,...,...
2025-11-17,293.970001,754.510010,607.830017,101.940002,999.989990,26013.449219,58962.699219,17444.500000,323.011581
2025-11-18,292.970001,749.599976,607.849976,100.910004,1000.000000,25910.050781,58899.250000,17337.650391,322.941763
2025-11-19,294.369995,749.409973,610.400024,102.230003,1000.000000,26052.650391,59216.050781,17376.800781,324.925813
